In [4]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install gensim.models

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement gensim.models (from versions: none)
ERROR: No matching distribution found for gensim.models


In [7]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [8]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aviro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
df = pd.read_csv('datasets/linux/Linux_2k.log_structured.csv')
df.head()

,LineId,Month,Date,Time,Level,Component,PID,Content,EventId,EventTemplate
0,1,Jun,14,15:16:01,combo,sshd(pam_unix),19939.0,authentication failure; logname= uid=0 euid=0 ...,E16,authentication failure; logname= uid=0 euid=0 ...
1,2,Jun,14,15:16:02,combo,sshd(pam_unix),19937.0,check pass; user unknown,E27,check pass; user unknown
2,3,Jun,14,15:16:02,combo,sshd(pam_unix),19937.0,authentication failure; logname= uid=0 euid=0 ...,E16,authentication failure; logname= uid=0 euid=0 ...
3,4,Jun,15,02:04:59,combo,sshd(pam_unix),20882.0,authentication failure; logname= uid=0 euid=0 ...,E18,authentication failure; logname= uid=0 euid=0 ...
4,5,Jun,15,02:04:59,combo,sshd(pam_unix),20884.0,authentication failure; logname= uid=0 euid=0 ...,E18,authentication failure; logname= uid=0 euid=0 ...


In [10]:
null_data = df.isnull().sum()

In [11]:
print(null_data[null_data > 0])

PID    151
dtype: int64


In [21]:
df['Tokens'] = df['Content'].apply(word_tokenize)
df.head()

,LineId,Month,Date,Time,Level,Component,PID,Content,EventId,EventTemplate,tokens,TimeInSeconds,Tokens
0,1,Jun,14,15:16:01,combo,sshd(pam_unix),19939.0,authentication failure; logname= uid=0 euid=0 ...,E16,authentication failure; logname= uid=0 euid=0 ...,"[authentication, failure, ;, logname=, uid=0, ...",54961,"[authentication, failure, ;, logname=, uid=0, ..."
1,2,Jun,14,15:16:02,combo,sshd(pam_unix),19937.0,check pass; user unknown,E27,check pass; user unknown,"[check, pass, ;, user, unknown]",54962,"[check, pass, ;, user, unknown]"
2,3,Jun,14,15:16:02,combo,sshd(pam_unix),19937.0,authentication failure; logname= uid=0 euid=0 ...,E16,authentication failure; logname= uid=0 euid=0 ...,"[authentication, failure, ;, logname=, uid=0, ...",54962,"[authentication, failure, ;, logname=, uid=0, ..."
3,4,Jun,15,02:04:59,combo,sshd(pam_unix),20882.0,authentication failure; logname= uid=0 euid=0 ...,E18,authentication failure; logname= uid=0 euid=0 ...,"[authentication, failure, ;, logname=, uid=0, ...",7499,"[authentication, failure, ;, logname=, uid=0, ..."
4,5,Jun,15,02:04:59,combo,sshd(pam_unix),20884.0,authentication failure; logname= uid=0 euid=0 ...,E18,authentication failure; logname= uid=0 euid=0 ...,"[authentication, failure, ;, logname=, uid=0, ...",7499,"[authentication, failure, ;, logname=, uid=0, ..."


In [22]:
from sklearn.preprocessing import LabelEncoder

# Prepare your tokenized sentences
sentences = df['tokens'].values.tolist()

# Flatten the list of tokenized words
words = [word for sentence in sentences for word in sentence]

# Convert words to numeric labels
label_encoder = LabelEncoder()
X = label_encoder.fit_transform(words)

# Reshape X for the HMM
X = X.reshape(-1, 1)
X = X.tolist()
##print(X)
lab = dict(zip(words, X))
print(lab)



{'authentication': [411], 'failure': [467], ';': [261], 'logname=': [511], 'uid=0': [661], 'euid=0': [463], 'tty=NODEVssh': [659], 'ruser=': [624], 'rhost=218.188.2.4': [591], 'check': [432], 'pass': [543], 'user': [669], 'unknown': [663], 'rhost=220-135-151-1.hinet-ip.hinet.net': [594], 'user=root': [671], 'session': [633], 'opened': [537], 'for': [471], 'cyrus': [446], 'by': [422], '(': [6], ')': [7], 'closed': [434], 'ALERT': [266], 'exited': [465], 'abnormally': [394], 'with': [682], '[': [391], '1': [66], ']': [392], 'news': [529], 'rhost=061092085098.ctinets.com': [570], 'rhost=d211-116-254-214.rev.krline.net': [609], 'connection': [441], 'from': [474], '24.54.76.216': [216], '24-54-76-216.bflony.adelphia.net': [215], 'at': [408], 'Fri': [306], 'Jun': [323], '17': [147], '07:07:00': [46], '2005': [170], '07:07:02': [47], '07:07:04': [48], 'rhost=211.46.224.253': [586], 'user=guest': [670], 'test': [647], 'uid=509': [662], '82.252.162.81': [252], 'lns-vlq-45-tou-82-252-162-81.adsl

In [27]:
# Convert Time to seconds for easier grouping
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s

# Helper function to convert seconds to hh:mm:ss
def seconds_to_hms(seconds):
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return f"{h:02}:{m:02}:{s:02}"

df['TimeInSeconds'] = df['Time'].apply(time_to_seconds)

# Step 4: Group Logs into Sequences by Time Window and Day
time_window = 5  # Define the time window in seconds
sequences = []
time_frames = []  # To store time frames for each sequence
current_sequence = []
current_start_time = None  # Track the start time of the current sequence
prev_time = None
prev_date = None
prev_month = None

for _, row in df.iterrows():
    is_same_day = (
        prev_date == row["Date"] and prev_month == row["Month"]
    )  # Check if the log is on the same day
    
    if (
        prev_time is None
        or (row["TimeInSeconds"] - prev_time <= time_window and is_same_day)
    ):
        if current_start_time is None:
            current_start_time = f"{row['Month']} {row['Date']} {row['Time']}"  # Start time of the new sequence
        current_sequence.append(row["Tokens"])
        prev_time = row["TimeInSeconds"]  # Update prev_time within the sequence
    else:
        sequences.append(current_sequence)
        end_time_hms = seconds_to_hms(prev_time)  # Convert end time to hh:mm:ss
        time_frames.append((current_start_time, f"{prev_month} {prev_date} {end_time_hms}"))  # Add time frame
        current_sequence = [row["Tokens"]]
        current_start_time = f"{row['Month']} {row['Date']} {row['Time']}"  # Reset the start time
        prev_time = row["TimeInSeconds"]  # Reset prev_time for the new sequence
    prev_date = row["Date"]
    prev_month = row["Month"]

# Add the last sequence and its time frame
if current_sequence:
    end_time_hms = seconds_to_hms(prev_time)
    time_frames.append((current_start_time, f"{prev_month} {prev_date} {end_time_hms}"))
    sequences.append(current_sequence)

# Visualization: Structured display with time frames
for i, (sequence, time_frame) in enumerate(zip(sequences, time_frames)):
    print(f"Sequence {i + 1} (Time Frame: {time_frame[0]} to {time_frame[1]}):")
    for log in sequence:
        print(f"  - {log}")
    print("\n")

Sequence 1 (Time Frame: Jun 14 15:16:01 to Jun 14 15:16:02):
  - ['authentication', 'failure', ';', 'logname=', 'uid=0', 'euid=0', 'tty=NODEVssh', 'ruser=', 'rhost=218.188.2.4']
  - ['check', 'pass', ';', 'user', 'unknown']
  - ['authentication', 'failure', ';', 'logname=', 'uid=0', 'euid=0', 'tty=NODEVssh', 'ruser=', 'rhost=218.188.2.4']


Sequence 2 (Time Frame: Jun 15 02:04:59 to Jun 15 02:04:59):
  - ['authentication', 'failure', ';', 'logname=', 'uid=0', 'euid=0', 'tty=NODEVssh', 'ruser=', 'rhost=220-135-151-1.hinet-ip.hinet.net', 'user=root']
  - ['authentication', 'failure', ';', 'logname=', 'uid=0', 'euid=0', 'tty=NODEVssh', 'ruser=', 'rhost=220-135-151-1.hinet-ip.hinet.net', 'user=root']
  - ['authentication', 'failure', ';', 'logname=', 'uid=0', 'euid=0', 'tty=NODEVssh', 'ruser=', 'rhost=220-135-151-1.hinet-ip.hinet.net', 'user=root']
  - ['authentication', 'failure', ';', 'logname=', 'uid=0', 'euid=0', 'tty=NODEVssh', 'ruser=', 'rhost=220-135-151-1.hinet-ip.hinet.net', 'user